In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:
from collections import defaultdict
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [4]:
meta_samples = pd.read_csv('../raw_metaSamples.tsv',sep="\t",index_col=0)
meta_samples.head()

,Tissue,Experiment,Name,Full
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC1,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC2,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,BMDM1hKLA,GRO,BMDM1hKLA_GRO1,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,BMDM1hKLA,GROCap,BMDM1hKLA_GROCap1,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...
CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq.gz,BMDMwt,GRO,BMDMwt_GRO1,CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001...


In [5]:
peaks_tissue = pd.read_csv("Results/merged/samples.merge.peaksexpression.log10",index_col=0,sep="\t")
peaks_tissue.head()

,f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,f04_peaks/Liver_START2_and_Liver_startinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,f04_peaks/Liver_START1_and_Liver_startinput1,f04_peaks/Lung_GROCap1_and_Lung_GRO1,f04_peaks/Brain_START3_and_Brain_startinput1,f04_peaks/Heart_START1_and_Heart_startinput1,f04_peaks/CHO_GROCap2_and_CHO_GRO1,f04_peaks/Heart_START2_and_Heart_startinput1,f04_peaks/Liver_GROCap1_and_Liver_GRO1,...,f04_peaks/MiscOrgans_START2_and_MiscOrgans_startinput1,f04_peaks/Spleen_START1_and_Spleen_startinput1,f04_peaks/Lung_START2_and_Lung_startinput1,f04_peaks/CHO_START1_and_CHO_startinput1,f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1,f04_peaks/Kidney_START2_and_Kidney_startinput1,f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,f04_peaks/CHO_GROCap1_and_CHO_GRO1,f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,f04_peaks/Spleen_START2_and_Spleen_startinput1
0,0.382017,0.0,0.356026,0.000000,0.876218,0.000000,0.426511,1.515874,0.0,0.000000,...,0.000000,0.000000,0.000000,0.804821,0.000000,0.0,0.868644,1.531479,0.000000,0.000000
1,0.472756,0.0,0.485721,0.633468,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.843855,0.309630,1.618048,0.000000,0.822822,0.0,0.000000,0.000000,0.000000,0.826723
2,0.382017,0.0,0.290480,0.000000,0.679428,0.000000,0.000000,1.372912,0.0,0.000000,...,0.000000,0.000000,0.000000,0.893762,0.434569,0.0,0.451786,1.354108,0.822168,0.545307
3,0.694605,0.0,0.000000,0.000000,0.000000,0.627366,0.426511,0.000000,0.0,0.000000,...,0.491362,0.648360,0.000000,0.617000,0.000000,0.0,0.000000,0.559907,0.536558,0.517196
4,2.220892,0.0,0.456366,1.918030,0.000000,1.648360,1.717671,0.000000,0.0,0.917506,...,1.776701,1.283301,0.000000,1.968016,0.000000,0.0,0.401401,0.761928,0.641474,2.203848


In [6]:
def merge_columns(df, mapping_dict):
    '''Function that merges columns by taking the mean of them.
    Merges based on if they have the same element in the meta_samples meta_column.
    Returns:
        new_df: Dataframe but with the columns of interest merged. Also column names are now based
                on the unique meta_samples[meta_column].
    '''

    vals = mapping_dict.keys()#np.unique(meta_samples[meta_column].values)
    new_df = pd.DataFrame(index=df.index, columns=vals)
    for i in vals:
        if not mapping_dict[i] == []:#(meta_samples[meta_column] == i).any():
            new_col = (df.loc[:, mapping_dict[i]])
            new_col = new_col.mean(axis=1)
            new_df.loc[:, i] = new_col
    new_df = new_df.loc[:, ~(new_df.isnull().all())]
    return new_df


In [7]:
tissue_to_peaks = defaultdict(list)
for val in peaks_tissue.columns.values:
    count = 0
    for t in np.unique(meta_samples["Tissue"].values):
        if t in val:
            count += 1
            tissue_to_peaks[t].append(val)
            if count > 1:
                print("This sample has multiple tissues associated with it")
                print("Sample: ",val)

In [8]:
peaks_tissue_merged = merge_columns(peaks_tissue,mapping_dict=tissue_to_peaks)

In [9]:
peaks_tissue_merged.head()

,Heart,Spleen,MiscOrgans,FemaleReproductive,CHO,BMDM1hKLA,Brain,Lung,Liver,BMDMwt,Muscle,Kidney,Pancreas
0,0.213256,0.000000,0.191009,0.177054,1.379905,1.92993,0.192445,0.292073,0.000000,0.000000,0.191908,0.289548,0.000000
1,0.000000,0.568176,0.658306,0.704141,0.000000,0.00000,0.365353,0.875407,0.211156,0.000000,0.160073,0.206016,0.000000
2,0.000000,0.272654,0.191009,0.649540,1.269561,0.00000,0.285237,0.439970,0.000000,0.822168,0.251895,0.377071,0.299942
3,0.213256,0.582778,0.592983,0.556972,0.347363,0.00000,0.633209,0.256951,0.000000,0.536558,0.300486,0.216445,0.380588
4,0.858835,1.743575,1.998797,1.219034,0.698374,0.00000,1.707648,0.729080,0.945179,0.641474,1.198274,0.838893,1.141311


In [10]:
peaks_tissue_merged.to_csv("Results/merged/tissues.merge.peaksexpression.log10",sep="\t")

## Do the same but for txn_df 

In [11]:
txn_df = pickle.load(open("Results/tss_annotation/txn_df_02.p","rb"))

In [12]:
## invert and create a peaks_to_tissue
peaks_to_tissue = dict()
for t in tissue_to_peaks:
    for p in tissue_to_peaks[t]:
        peaks_to_tissue[os.path.basename(p)] = t

In [13]:
txn_df["Tissues"] = ""
for ind,val in tqdm.tqdm(txn_df.iterrows()):
    if len(val["samples"]) != 0:
        tis = set()
        for t in val["samples"]:
            tis.add(peaks_to_tissue[t])
        txn_df.at[ind,"Tissues"] = ",".join(tis)

52382it [00:13, 3759.35it/s]


In [14]:
pickle.dump(txn_df,open("Results/tss_annotation/txn_df_02_tissues.p","wb"))


In [15]:
txn_df.columns

Index([u'peaks', u'closest_to_X_numberOfPeaks', u'bin_-1000_1000',
       u'sameStrand_bin_-1000_1000', u'samples', u'minDistance',
       u'minDistancePeak', u'maxValue', u'maxPeakId', u'maxSample', u'Chr',
       u'Start', u'End', u'Strand', u'gene_id', u'gene', u'transcript',
       u'maxSamples', u'hasGene', u'Number of SS', u'Tissues'],
      dtype='object')

In [16]:
txn_df[["samples","Tissues"]]

,samples,Tissues
asmbl_2,[],
asmbl_4,[CHO_START1_and_CHO_startinput1],CHO
asmbl_7,[],
asmbl_8,[],
asmbl_9,[],
asmbl_10,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...","Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."
asmbl_11,[],
asmbl_12,[],
asmbl_14,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...","Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."
asmbl_15,[],
